<h4><center> Introduction to building a quant pricing library practice
 </center></h4>

<h1><center> Pricing Barrier Options </center></h1>

<h4><center>
<h3><center> Varga László </center></h3> <br /> 
Department of Probability Theory and Statistics, ELTE, TTK <br /> 
</center></h4>
     
<h4><center> <a href="mailto:vargala4@gmail.com">vargala4@gmail.com</a>  </center></h4>    

<h3><center> Introduction - Barrier options </center></h3>

- Derivative which payoff depends on whether the underlier's price has reached a pre-determined barrier value
- Payoff is path-dependent
- Exercise is usually European style





- Advantages:
    - A wisely chosen barrier option may provide additional protection, therefore they are very popolar tools for hedging
    - Barrier option prices are smaller than the prices of the respective vanilla options


- Pricing methods:
    - exact calculation: available only for very few types
    - **Monte Carlo simulation** - most popular
        - Brownian bridges technique - for increased efficiency
    - numerical solution of the pricing PDE


<h3><center> Mathematical notations </center></h3>

- $(S_t)_{t\in [0,T]}$: asset price process
- $T$: exercise date
- $K$: strike
- $\{ t_1,\ldots,t_n \}$: fix observation times, $0\leq t_1 \leq \ldots \leq t_n \leq T$ on which we take the average of the asset price
- $A_T$: average of the underlying asset prices in time interval $[0,T]$
- $x_+=max(x,0)$: positive value of $x$

<h3><center> Types of barrier options </center></h3>

- Call or Put
- Knock-in or knock-out
- **Single** (upper or lower) or double (upper and lower) barrier
- Continuously or discretely monitored
    


- Averaging type and Monitoring frequency

**Averaging type $\quad$  Continuously monitored $\qquad \qquad \ $ Discretely monitored**                                                    
Geometric    $\qquad$    $\left(exp\left\{ \frac{1}{T}\int\limits_0^T \log S_t \,dt \right\} -K \right)_+$    $\quad$      $\left(  \left( \prod\limits_{i=1}^n S_{t_i} \right)^{1/n} -K \right)_+ $

Arithmetic  $\qquad \qquad$  $\left( \frac{1}{T}\int\limits_0^T S_t\, dt-K \right)_+$ $\qquad \qquad \quad$ $\color{blue}{\left( \frac{1}{n} \sum\limits_{i=1}^n S_{t_i} -K \right)_+} $



<h3><center> Pricing of Asian options </center></h3>

- Price (PV) of a Call Asian option at time $t=0: \quad e^{-rT}E\left[ (A_T-K)_+ \right]$
- There is no closed formula for the price, as $A_T$ is the arithmetical average of lognormal random variables, which is (usually) not lognormally distributed

<h3><center> Pricing by moment matching </center></h3>

- First proposed by Edmond Levy for Asian options
- We approximate $A_T$ by a properly parametrized lognormal random variable $V$
- The lognormal distribution is determined by its first two moments, therefore we match the first two moments of $A_T$ and $V$


First and second moments of $A_T$ are
- $ E \left(A_T \right) =  \frac{S_0}{n}\sum\limits_{i=1}^{n}\exp\left\{rt_i\right\}$
- $ E \left( A_T^2 \right) =  \frac{S^2_0}{n^2}\sum\limits_{i,j=1}^{n}\exp\left\{r(t_i+t_j) + \sigma^2\min(t_i, t_j)\right\}$

$\hspace{-5mm}$
If $V$ is lognormal with parameters $m$ and $\sigma$, then the first two moments of $V$ are 

$\quad EV = \exp\left\{m+\frac{\sigma^2}{2} \right\} \quad $ and 
$\quad E V^2 = \exp\left\{2m+2\sigma^2 \right\}$

Matching the first two moments and solving the system of equations:
- $m = 2\, \text{log} E \! \left(A_T\right) - \frac{1}{2} \, \text{log} E \! \left( A_T^2 \right)$ 
- $\sigma^2 = \text{log} E \! \left( A_T^2 \right) - 2 \text{log} E \! \left( A_T \right)$

Finally we can substitute into the Black-Scholes formula to get the moment matching approximation price of a Call Asian option:

$\hspace{30mm} e^{-rT}\left(e^{m+\frac{\sigma^2}{2}} \Phi\left(d_1\right)-K \Phi\left(d_2\right)\right)$, where

- $d_1=\frac{\log \left(\frac{S_0}{K}\right)+m+\sigma^2}{\sigma}$
- $d_2=d_1-\sigma$ 
    

<h3><center> Simulations </center></h3>

- Showing the Asian pricer in the codebase and do some pricing exercise.

- Comparison of the moment matching approach with Monte Carlo pricing. Relative and absolute error.



In [ ]:
direction

In [1]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

current = Path().resolve()
sys.path.append(str(current))
sys.path.append(str(current.parents[1]))

from src.enums import *
from src.utils import *
from src.market_data import *
from src.pricer import *
# Make charts interactive
%matplotlib notebook

# Initialize market data
MarketData.initialize()

100.0
0.3
4.83847947905555


In [2]:
underlying = Stock.TEST_COMPANY
spot = FlatVolModel(underlying).get_spot()
print(spot)

100.0


In [3]:
moneyness = 1
expiry = 1
strike = spot * moneyness
vol = FlatVolModel(underlying).get_vol(strike, expiry)
print(vol)

0.3


In [15]:
nr_monitoring_points = 100 
barrier = 90
up_down = UpDown.DOWN
in_out = InOut.IN

volgrid = MarketData.get_vol_grid()[underlying]
contract = EuropeanBarrierContract(underlying, PutCallFwd.CALL, LongShort.LONG, strike, expiry, 
                                   nr_monitoring_points, barrier, up_down, in_out)
model = FlatVolModel(underlying)
params = Params()
pricer_AN = BarrierAnalyticPricer(contract, model, params)
print(pricer_AN.calc_fair_value())

# params_MC = MCParams(seed = 2)
params_MC = MCParams(seed = 9, num_of_path = 10000, tenor_frequency = 100)
pricer_MC = GenericMCPricer(contract, model, params_MC)
print(pricer_MC.calc_fair_value())

4.83847947905555
(3.974237896431247, [3.7579589221183074, 4.190516870744187])


In [16]:
contract.get_barrier().get_barrier_level()

90

In [18]:
contract.get_barrier().get_in_out()

<InOut.IN: 'IN'>

In [19]:
contract.get_barrier().get_up_down()

<UpDown.DOWN: 'DOWN'>

<h3><center> Assignment </center></h3>

- Show by simulation and a graph that for increasing volatility, the error of the moment matching approximation increases.
- Show by simulation and a graph that for longer averaging periods, the error of the moment matching approximation increases. What is the mathematical theorem that substitutes this result? (central limit theorem)

<h3><center> Conclusions </center></h3>

Experience with the moment matching technique
- Fast calculations
- The approximation is very good for short averaging periods
- The approximation gets worse if volatility is large and the averaging period is long
- Risks are stable even under severely stressed conditions

<h3><center> References </center></h3>

- Pak Keung Chan: Pricing Asian Options by the Method of Moments Matching. Master Thesis, University of Waterloo, Ontario, Canada, 2015
- Edmond Levy: Pricing european average rate currency options. Vol. 11, No 5. (1992), pp. 474–491, *Journal of International Money and Finance*
- Stuart M. Turnbull and Lee MacDonald Wakeman: A Quick Algorithm for Pricing European Average Options. The Journal of Financial and Quantitative Analysis. Vol. 26, No. 3 (Sep., 1991), pp. 377-389, *Cambridge University Press*
